In [1]:
from __future__ import annotations # To prevent Subscript for class "list" will generate runtime exception; enclose type annotation in quotes
import seaborn as sns
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import scipy
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from Kmeans import *
from decimal import Decimal
from SVD import * 
%matplotlib inline
# pd.options.display.max_rows = 4000
# pd.options.display.max_columns = 4000

In [2]:
df = pd.read_csv('preprocessed.csv',index_col=0)
Genres=list(pd.read_csv('genre_order.csv',index_col=0)['0'])
df

,Gender,Age,Zip-code_1,Zip-code_2,Zip-code_3,Zip-code_4,Zip-code_5,Zip-code_6,Zip-code_7,Zip-code_8,...,Action,1960s,Animation,Comedy,Thriller,Mystery,1940s,Children's,Film-Noir,Drama
UserID,,,,,,,,,,,,,,,,,,,,,
1,0,0.017857,0,0,0,1,0,0,0,0,...,0.182449,0.148240,0.645186,0.504759,0.091603,7.379219e-02,0.114032,0.747438,0.004701,0.822336
2,1,1.000000,0,0,0,0,0,0,1,0,...,0.607944,0.114536,0.027614,0.273131,0.330404,2.202718e-02,0.017621,0.057179,0.004405,1.000000
3,1,0.446429,0,0,0,0,1,0,0,0,...,0.814808,0.061729,0.086422,1.000000,0.148147,6.255649e-07,0.154653,0.086422,0.125994,0.271601
4,1,0.803571,0,1,0,0,0,0,0,0,...,0.354289,0.078345,0.194632,1.000000,0.047685,1.704221e-01,0.144273,0.004760,0.104962,0.102874
5,1,0.446429,0,0,0,0,1,0,0,0,...,0.125002,0.000000,0.026040,0.346350,0.182293,3.906287e-02,0.005208,0.039062,0.018228,0.562499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0,0.446429,0,0,1,0,0,0,0,0,...,0.294317,0.111286,0.091950,0.552734,0.301040,7.549845e-02,0.051283,0.123925,0.046457,0.898209
6037,0,0.803571,0,0,0,0,0,0,1,0,...,0.257441,0.253624,0.003553,0.535784,0.650177,1.178472e-01,0.135690,0.049999,0.071433,1.000000
6038,0,1.000000,1,0,0,0,0,0,0,0,...,0.062499,0.000002,0.187502,1.000000,0.794578,2.552067e-01,0.403411,0.000000,0.134377,0.750100


In [3]:
def cluster_to_matrix(X_train,y_train,num_cluster):
    k=Kmeans(K=num_cluster,iter=150,p=2,random_state=42)
    y_pred,_=k.predict(np.array(X_train),choice=0)
    k.centroids=np.array(k.centroids)
    empty_cluster = []
    for idx,i in enumerate(k.clusters):
        if len(i)==0:
            empty_cluster.append(idx)
    X_train2 = pd.concat([X_train,y_train,pd.DataFrame(y_pred,columns=["label"],dtype='int')],axis="columns")
    df_cluster_rep = X_train2.groupby(['label'], as_index=False).mean()
    df_cluster_rep.drop(labels="label",axis="columns",inplace=True)
    matrix = df_cluster_rep.to_numpy()
    final_matrix = []
    j = 0 
    for i in range(num_cluster):
        if i in empty_cluster:
            final_matrix.append(np.zeros(len(df.columns)))
        else:
            final_matrix.append(matrix[j])
            j+=1

    return np.array(final_matrix),k 

In [4]:
x_col_size=len(df.columns)-len(Genres)+6

In [5]:
x_col_size

38

In [6]:
y = df[df.columns[x_col_size:]]
x = df.drop(labels=df.columns[x_col_size:],axis="columns")

In [7]:
x

,Gender,Age,Zip-code_1,Zip-code_2,Zip-code_3,Zip-code_4,Zip-code_5,Zip-code_6,Zip-code_7,Zip-code_8,...,Occupation_17,Occupation_18,Occupation_19,Occupation_20,Documentary,Sci-Fi,Musical,1910s,War,1970s
UserID,,,,,,,,,,,,,,,,,,,,,
1,0,0.017857,0,0,0,1,0,0,0,0,...,0,0,0,0,0.020804,0.114028,0.527754,0.025357,0.091980,7.981933e-02
2,1,1.000000,0,0,0,0,0,0,1,0,...,0,0,0,0,0.021639,0.185026,0.054848,0.000985,0.171806,8.810512e-02
3,1,0.446429,0,0,0,0,1,0,0,0,...,0,0,0,0,0.117177,0.185184,0.012347,0.002761,0.049383,1.975300e-01
4,1,0.803571,0,1,0,0,0,0,0,0,...,0,0,0,0,0.037577,0.127402,0.181496,0.000000,0.029289,9.674175e-02
5,1,0.446429,0,0,0,0,1,0,0,0,...,0,0,0,1,0.036459,0.075522,0.013020,0.000583,0.033854,1.436880e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0,0.446429,0,0,1,0,0,0,0,0,...,0,0,0,0,0.027099,0.296310,0.077436,0.000000,0.109364,1.626059e-01
6037,0,0.803571,0,0,0,0,0,0,1,0,...,0,0,0,0,0.003553,0.368582,0.035691,0.003553,0.164419,3.536907e-01
6038,0,1.000000,1,0,0,0,0,0,0,0,...,0,0,0,0,0.187507,0.031249,0.430235,0.031252,0.312534,2.187714e-01


In [8]:
y

,Western,Horror,1980s,1950s,1990s,Crime,Romance,Fantasy,2000s,Adventure,...,Action,1960s,Animation,Comedy,Thriller,Mystery,1940s,Children's,Film-Noir,Drama
UserID,,,,,,,,,,,,,,,,,,,,,
1,0.061512,0.176776,0.364898,0.125432,1.000000,0.068418,0.183203,0.102628,0.034208,0.171613,...,0.182449,0.148240,0.645186,0.504759,0.091603,7.379219e-02,0.114032,0.747438,0.004701,0.822336
2,0.035242,0.008810,0.259911,0.061674,0.911911,0.127754,0.277536,0.000000,0.008810,0.220268,...,0.607944,0.114536,0.027614,0.273131,0.330404,2.202718e-02,0.017621,0.057179,0.004405,1.000000
3,0.246911,0.037037,0.703702,0.205044,0.740736,0.507604,0.148148,0.061728,0.000000,0.901228,...,0.814808,0.061729,0.086422,1.000000,0.148147,6.255649e-07,0.154653,0.086422,0.125994,0.271601
4,0.029289,0.047685,0.090609,0.205070,0.072213,0.010893,0.023157,0.029289,0.114861,0.090609,...,0.354289,0.078345,0.194632,1.000000,0.047685,1.704221e-01,0.144273,0.004760,0.104962,0.102874
5,0.002604,0.041665,0.020832,0.013019,1.000000,0.119792,0.158857,0.050074,0.023437,0.041665,...,0.125002,0.000000,0.026040,0.346350,0.182293,3.906287e-02,0.005208,0.039062,0.018228,0.562499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.031936,0.138414,0.435581,0.113240,1.000000,0.125842,0.273937,0.032923,0.006779,0.155877,...,0.294317,0.111286,0.091950,0.552734,0.301040,7.549845e-02,0.051283,0.123925,0.046457,0.898209
6037,0.032161,0.092925,0.368009,0.246573,0.496549,0.174995,0.203597,0.039324,0.028320,0.089441,...,0.257441,0.253624,0.003553,0.535784,0.650177,1.178472e-01,0.135690,0.049999,0.071433,1.000000
6038,0.306259,0.031251,0.437526,0.156252,0.562553,0.434890,0.531311,0.205366,0.385416,0.031249,...,0.062499,0.000002,0.187502,1.000000,0.794578,2.552067e-01,0.403411,0.000000,0.134377,0.750100


In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.05, shuffle=True, random_state=42)

In [10]:
num_cluster=25

In [11]:
matrix,kmeans_model = cluster_to_matrix(x_train,y_train,num_cluster)

In [12]:
matrix.shape

(25, 60)

In [13]:
def predict(datapoint,kmeans,matrix_clean):
    ''' 
        @input: Takes in a datapoint, kmeans object, and information matrix 
        @output: Returns: rating sorted with index for column name mapping
    '''
    y_pred = kmeans.predictPoint(datapoint)
    inferred_full = matrix_clean[y_pred]
    inferred = inferred_full[x_col_size:]
    inferred_full = inferred_full[len(df.columns)-len(Genres):]
    sorted_ratings = []
    sorted_ratings_full = []
    for idx,element in enumerate(inferred):
        sorted_ratings.append([element,idx+x_col_size])
    for idx,element in enumerate(inferred_full):
        sorted_ratings_full.append([element,idx+len(df.columns)-len(Genres)])
    sorted_ratings.sort()
    sorted_ratings.reverse()
    sorted_ratings_full.sort()
    sorted_ratings_full.reverse()
    return sorted_ratings, sorted_ratings_full

In [14]:
x_test.to_numpy()[0][:x_col_size]
sorted_ratings, sorted_ratings_full = predict(x_test.to_numpy()[0][:x_col_size],kmeans_model,matrix)
s="Top 5 PREDICTED recommended genres are:\n"
for i in range(5):
    s+=df.columns[sorted_ratings_full[i][1]]+"\n"
print(s)

Top 5 PREDICTED recommended genres are:
1990s
Drama
Comedy
Action
1980s



In [38]:
sorted_ratings_actual=[]
inferred = y_test.to_numpy()[0]
for idx,element in enumerate(inferred):
    sorted_ratings_actual.append([element,idx+x_col_size])
sorted_ratings_actual.sort()
sorted_ratings_actual.reverse()
s="Top 5 ACTUAL recommended genres are:\n"
for i in range(5):
    s+=df.columns[sorted_ratings_actual[i][1]]+"\n"
print(s)

Top 5 ACTUAL recommended genres are:
1990s
Drama
Comedy
1980s
Romance



In [16]:
rating_dict={}
for i in range(len(sorted_ratings_full)):
    rating_dict[df.columns[sorted_ratings_full[i][1]]]=sorted_ratings_full[i][0]
rating_dict

{'1990s': 0.7554963533967542,
 'Drama': 0.5825798651585476,
 'Comedy': 0.5444009488733148,
 'Action': 0.4090980846327718,
 '1980s': 0.34362257371114224,
 'Thriller': 0.3218229116595851,
 'Romance': 0.2616338634296228,
 'Sci-Fi': 0.24405736941749598,
 'Adventure': 0.23236766429914238,
 '1970s': 0.21398053006341694,
 'Crime': 0.17115603770793736,
 '1960s': 0.16231688680032616,
 '1950s': 0.15746546763323704,
 '2000s': 0.15258824439190105,
 'War': 0.15222408880082178,
 '1940s': 0.13260858830402458,
 "Children's": 0.12623656733647765,
 'Mystery': 0.12030291736542044,
 'Horror': 0.11990518063940378,
 'Animation': 0.11623909173410132,
 'Musical': 0.11457736784161415,
 'Film-Noir': 0.0932768698904159,
 '1930s': 0.09112021915999073,
 'Fantasy': 0.08964294596876471,
 'Western': 0.08415213805308672,
 'Documentary': 0.07025097602471758,
 '1920s': 0.04545067175289975,
 '1910s': 0.009576197203511739}

In [17]:
# def predict_for_dataset(X_test,kmeans_instance,matrix_clean):
#     X_test = X_test.to_numpy()
#     y_pred = []
#     for i in range(X_test.shape[0]):
#         datapoint =X_test[i][:x_col_size]
#         y_pred.append(predict(datapoint,kmeans_instance,matrix_clean))
#     return np.array(y_pred)

In [18]:
# y_pred = predict_for_dataset(x_test,kmeans_model,matrix) 
# y_pred.shape

In [19]:
# def accuracy(y_pred,y_test):
#     y_test = y_test.to_numpy()
#     sorted_ratings_actual = []
#     for i in range(y_test.shape[0]):
#         inferred = y_test[i]
#         t = []
#         for idx,element in enumerate(inferred):
#             t.append([element,idx+x_col_size])
#         t.sort()
#         t.reverse()
#         sorted_ratings_actual.append(np.array(t))
#     sorted_ratings_actual = np.array(sorted_ratings_actual)
#     # print(sorted_ratings_actual.shape)
    
#     total = 0 
#     correct = 0 
    
#     for i in range(sorted_ratings_actual.shape[0]):
#         total+=x_col_size
#         temp = sorted_ratings_actual[i]
#         for j in range(y_pred.shape[1]):
#             for k in range(len(temp)):
#                 # print(i,j,k)
#                 if(y_pred[i][j][1]==temp[k][1]):
#                     correct+=1
#                     break

#     percent_correct_for_datapoint = []

#     for i in range(sorted_ratings_actual.shape[0]):
#         per_correct = 0 
#         temp = sorted_ratings_actual[i][:x_col_size]
#         for j in range(y_pred.shape[1]):
#             for k in range(len(temp)):
#                 if(y_pred[i][j][1]==temp[k][1]):per_correct+=1
#         percent_correct_for_datapoint.append(per_correct/x_col_size)
#     # print("Total Entries received: ", len(sorted_ratings_actual)*5," Total Correct: ",correct)
#     return 100*(correct/total),percent_correct_for_datapoint,sorted_ratings_actual

In [20]:
# overall_val,per_point_val,y_sorted_actual = accuracy(y_pred,y_test)

In [21]:
# overall_val

In [22]:
ratings=pd.read_csv('ml-1m/ratings.dat', delimiter='::', names=['UserID','MovieID','Rating','TimeStamp'])
movies=pd.read_csv('ml-1m/movies.dat', delimiter='::', names=["Title","Genres"],  encoding="ISO-8859-1")

/tmp/ipykernel_13224/2496518303.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings=pd.read_csv('ml-1m/ratings.dat', delimiter='::', names=['UserID','MovieID','Rating','TimeStamp'])
/tmp/ipykernel_13224/2496518303.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies=pd.read_csv('ml-1m/movies.dat', delimiter='::', names=["Title","Genres"],  encoding="ISO-8859-1")


In [23]:
# movies
ratings.drop(['UserID','TimeStamp'], axis=1, inplace= True)

In [24]:
ratings

,MovieID,Rating
0,1193,5
1,661,3
2,914,3
3,3408,4
4,2355,5
...,...,...
1000204,1091,1
1000205,1094,5
1000206,562,5
1000207,1096,4


In [25]:
movies['Rating']=[[] for _ in range(len(movies.index))]
for i in ratings.index:
    temp=ratings.iloc[i]['Rating']
    movies.at[ratings.iloc[i]['MovieID'],'Rating'].append(temp)

In [26]:
movies['Rating']=movies['Rating'].apply(lambda x: np.array(x).mean())

/tmp/ipykernel_13224/1822539475.py:1: RuntimeWarning: Mean of empty slice.
  movies['Rating']=movies['Rating'].apply(lambda x: np.array(x).mean())
/home/netradeepakc/anaconda3/envs/rec_sys_mini/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [27]:
movies.isna().sum()

Title       0
Genres      0
Rating    177
dtype: int64

In [28]:
movies['Genres']=movies['Genres'].apply(lambda x: x.split('|'))
for i in movies.index:
    title=movies.at[i,'Title']
    decade=title[-5:-2]
    decade=str(decade)+'0s'
    movies.at[i,'Genres'].append(decade)

In [29]:
movies

,Title,Genres,Rating
1,Toy Story (1995),"[Animation, Children's, Comedy, 1990s]",4.146846
2,Jumanji (1995),"[Adventure, Children's, Fantasy, 1990s]",3.201141
3,Grumpier Old Men (1995),"[Comedy, Romance, 1990s]",3.016736
4,Waiting to Exhale (1995),"[Comedy, Drama, 1990s]",2.729412
5,Father of the Bride Part II (1995),"[Comedy, 1990s]",3.006757
...,...,...,...
3948,Meet the Parents (2000),"[Comedy, 2000s]",3.635731
3949,Requiem for a Dream (2000),"[Drama, 2000s]",4.115132
3950,Tigerland (2000),"[Drama, 2000s]",3.666667
3951,Two Family House (2000),"[Drama, 2000s]",3.900000


In [30]:
movies['Affinity']=[0 for _ in range(len(movies.index))]
for i in movies.index:
    genres=movies.at[i,'Genres']
    affinity=0
    for j in genres:
        affinity+=rating_dict[j]*movies.at[i,'Rating']
    movies.at[i,'Affinity']=affinity

/tmp/ipykernel_13224/3621030468.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.395983782391429' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  movies.at[i,'Affinity']=affinity


In [31]:
movies.sort_values('Affinity',inplace=True,ascending=False)

In [32]:
movies

,Title,Genres,Rating,Affinity
3266,Man Bites Dog (C'est arrivé près de chez vous)...,"[Action, Comedy, Crime, Drama, 1990s]",3.655844,9.003362
1215,Army of Darkness (1993),"[Action, Adventure, Comedy, Horror, Sci-Fi, 19...",3.804233,8.769995
293,"Professional, The (a.k.a. Leon: The Profession...","[Crime, Drama, Romance, Thriller, 1990s]",4.106176,8.592948
1196,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Drama, Sci-Fi, War, 1980s]",4.292977,8.431190
838,Emma (1996),"[Comedy, Drama, Romance, 1990s]",3.880240,8.319664
...,...,...,...,...
3650,Anguish (Angustia) (1986),"[Horror, 1980s]",NaN,NaN
3750,Boricua's Bond (2000),"[Drama, 2000s]",NaN,NaN
3829,Mad About Mambo (2000),"[Comedy, Romance, 2000s]",NaN,NaN
3856,Autumn Heart (1999),"[Drama, 1990s]",NaN,NaN


In [33]:
top_movies=list(movies['Title'])[:5]
print("Top 5 movies are:")
for i in top_movies:
    print(i)

Top 5 movies are:
Man Bites Dog (C'est arrivé près de chez vous) (1992)
Army of Darkness (1993)
Professional, The (a.k.a. Leon: The Professional) (1994)
Star Wars: Episode V - The Empire Strikes Back (1980)
Emma (1996)
